In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import widgets, interact, IntSlider, IntText, VBox, HBox
from matplotlib import rcParams
from IPython.display import display, clear_output
import warnings

# ========== 配置部分 ==========
# 1. 中文显示配置
try:
    rcParams['font.sans-serif'] = ['SimHei']  # Windows系统
    rcParams['axes.unicode_minus'] = False
except:
    print("警告：中文字体配置失败，请手动安装SimHei字体")

# 2. 数据加载
# 加载评分数据
ratings = pd.read_csv('E:/AAAAAAStudy/Study/大三下/python-web/work/work2/ml-1m/ratings.dat', sep='::', engine='python', 
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])

# 加载电影数据
movies = pd.read_csv('E:/AAAAAAStudy/Study/大三下/python-web/work/work2/ml-1m/movies.dat', sep='::', engine='python', 
                     names=['movie_id', 'title', 'genres'], encoding='latin-1')

# 3. 创建用户-物品矩阵
print("创建用户-物品矩阵...")
user_item_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

# 4. 计算用户相似度矩阵
print("计算用户相似度矩阵（可能需要1-2分钟）...")
user_similarity = cosine_similarity(user_item_matrix.values)
user_similarity_df = pd.DataFrame(user_similarity, 
                                 index=user_item_matrix.index, 
                                 columns=user_item_matrix.index)

# ========== 核心功能 ==========
def usercf_recommend(user_id, top_n=10, k=20):
    """用户协同过滤推荐核心算法"""
    try:
        similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:k+1]
        similar_users_ratings = user_item_matrix.loc[similar_users.index]
        weighted_ratings = similar_users_ratings.mul(similar_users, axis=0)
        predicted_ratings = weighted_ratings.sum(axis=0) / similar_users.sum()
        user_rated_items = user_item_matrix.loc[user_id]
        return predicted_ratings[user_rated_items == 0].sort_values(ascending=False).head(top_n)
    except KeyError:
        return pd.Series()

# ========== 可视化模块 ==========
def plot_matrix(matrix, title, figsize=(16, 12), cmap="YlGnBu"):
    """通用矩阵热力图可视化"""
    plt.figure(figsize=figsize)
    sns.heatmap(matrix, cmap=cmap, cbar_kws={'label': '数值'}, annot=False)
    plt.title(title, fontsize=14)
    plt.xlabel("列索引" if "物品" in title else "用户ID", fontsize=10)
    plt.ylabel("行索引" if "物品" in title else "用户ID", fontsize=10)
    plt.show()

def interactive_similarity_plot(user_id=1, top_users=20):
    """交互式相似度矩阵"""
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:top_users+1]
    subset = user_similarity_df.loc[similar_users, similar_users]
    
    fig = go.Figure(data=go.Heatmap(
        z=subset.values,
        x=subset.columns,
        y=subset.index,
        colorscale='Viridis',
        hoverongaps=False,
        hovertemplate="用户 %{x}<br>与用户 %{y} 的相似度: %{z:.2f}<extra></extra>"
    ))
    
    fig.update_layout(
        title=f'Top {top_users} 相似用户矩阵 (目标用户: {user_id})',
        width=800,
        height=700,
        xaxis_title="用户ID",
        yaxis_title="用户ID"
    )
    fig.show()

def plot_recommendations(user_id, top_n=10, k=20):
    recommendations = usercf_recommend(user_id, top_n, k)
    
    if recommendations.empty:
        print("无推荐结果")
        return
    
    rec_movies = movies[movies.movie_id.isin(recommendations.index)].copy()
    rec_movies["pred_rating"] = recommendations.values
    
    # Matplotlib静态可视化（修复版本）
    plt.figure(figsize=(12, 8))
    ax = sns.barplot(
        x="pred_rating", 
        y="title", 
        hue="title",  # 添加hue参数
        data=rec_movies.sort_values("pred_rating", ascending=False),
        palette="viridis",
        dodge=False,
        legend=False  # 关闭图例
    )
    plt.title(f"用户 {user_id} 的推荐结果", fontsize=14)
    plt.xlabel("预测评分", fontsize=10)
    plt.ylabel("")
    plt.tight_layout()
    plt.show()
    
    # ... 其余Plotly代码保持不变 ...
    
    # Plotly交互式可视化
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=rec_movies["pred_rating"],
        y=rec_movies["title"],
        orientation='h',
        marker_color='#1f77b4',
        hovertext=rec_movies.apply(lambda x: f"{x.title}<br>类型：{x.genres}<br>预测评分：{x.pred_rating:.2f}", axis=1),
        hoverinfo="text"
    ))
    
    fig.update_layout(
        title=f"交互式推荐结果 (用户 {user_id})",
        height=600,
        width=800,
        yaxis={'categoryorder':'total ascending'}
    )
    fig.show()

def explain_recommendations(user_id, movie_id):
    """推荐原因的可视化解释"""
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:6]
    user_ratings = user_item_matrix.loc[similar_users, movie_id]
    movie_info = movies[movies.movie_id == movie_id].iloc[0]
    
    # 创建解释仪表盘
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{'type':'xy'}, {'type':'domain'}],
               [{'colspan': 2}, None]],
        subplot_titles=(
            "相似用户评分分布", 
            "电影类型构成",
            "推荐原因摘要"
        )
    )
    
    # 评分分布
    fig.add_trace(go.Bar(
        x=user_ratings.index,
        y=user_ratings.values,
        name='用户评分',
        marker_color='lightseagreen'
    ), row=1, col=1)
    
    # 类型分布
    genres = movie_info['genres'].split('|')
    fig.add_trace(go.Pie(
        labels=genres,
        values=[1]*len(genres),
        hole=.3,
        name="类型分布"
    ), row=1, col=2)
    
    # 文本说明
    explanation_text = f"""
    <b>推荐逻辑说明：</b><br>
    1. 该推荐基于与用户 {user_id} 最相似的前5位用户<br>
    2. 相似用户的平均评分：{user_ratings.mean():.2f}<br>
    3. 电影类型包含：{', '.join(genres)}<br>
    4. 推荐权重计算方式：加权余弦相似度
    """
    fig.add_trace(go.Scatter(
        x=[0], y=[0],
        mode="text",
        text=explanation_text,
        textfont_size=12,
        showlegend=False
    ), row=2, col=1)
    
    fig.update_layout(
        title=f"推荐原因分析：{movie_info['title']}",
        height=800,
        width=1000,
        showlegend=False
    )
    fig.show()

# ========== 交互界面 ==========
class RecommendationDashboard(widgets.VBox):
    def __init__(self):
        super().__init__()
        
        # 控件初始化
        self.user_id = widgets.IntText(value=1, description="用户ID:", min=1, max=6040)
        self.top_n = widgets.IntSlider(value=5, min=1, max=20, description="推荐数量:")
        self.k_neighbors = widgets.IntSlider(value=15, min=5, max=50, description="相似用户数:")
        self.matrix_size = widgets.IntSlider(value=20, min=10, max=100, description="矩阵显示规模:")
        
        # 控件布局
        controls = VBox([
            HBox([self.user_id, self.top_n]),
            HBox([self.k_neighbors, self.matrix_size])
        ])
        
        self.output = widgets.Output()
        self.children = [controls, self.output]
        
        # 事件绑定
        self.user_id.observe(self.update_dashboard, names='value')
        self.top_n.observe(self.update_dashboard, names='value')
        self.k_neighbors.observe(self.update_dashboard, names='value')
        self.matrix_size.observe(self.update_dashboard, names='value')

    def update_dashboard(self, change):
        with self.output:
            clear_output(wait=True)
            user_id = self.user_id.value
            
            try:
                # 显示用户-物品矩阵
                plot_matrix(user_item_matrix.iloc[:self.matrix_size.value, :50], 
                           f"用户-物品评分矩阵 (前{self.matrix_size.value}用户)")
                
                # 显示相似度矩阵
                interactive_similarity_plot(user_id, self.k_neighbors.value)
                
                # 生成推荐结果
                plot_recommendations(user_id, self.top_n.value, self.k_neighbors.value)
                
                # 显示推荐解释
                recs = usercf_recommend(user_id, 1, self.k_neighbors.value)
                if not recs.empty:
                    explain_recommendations(user_id, recs.index[0])
                    
            except Exception as e:
                print(f"错误发生：{str(e)}")

# ========== 启动界面 ==========
print("系统初始化完成！")
dashboard = RecommendationDashboard()
dashboard

创建用户-物品矩阵...
计算用户相似度矩阵（可能需要1-2分钟）...
系统初始化完成！


RecommendationDashboard(children=(VBox(children=(HBox(children=(IntText(value=1, description='用户ID:'), IntSlid…